In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import re
import os

In [ ]:
# next page
# 'https://eda.ru/recepty/vypechka-deserty?page='+number
# 821 pages

In [4]:
def getInfo_edaru(filename, url, startPage, endPage):
    with open(filename, 'a') as file:
        writer = csv.writer(file)
        if os.stat(filename).st_size == 0:
            writer.writerow(['img', 'title', 'link', 'stepbystep', 'cuisine',
                            'portions', 'time', 'ingredients', 'favourite', 'likes', 'dislikes'])
            
        # walk the pages
        for page in range(startPage, endPage+1):
            pageLink = url+'?page='+str(page)
            soup = BeautifulSoup(requests.get(pageLink).text, 'html.parser')

            # extract the elements
            for element in soup.find_all('div', class_="tile-list__horizontal-tile horizontal-tile js-portions-count-parent js-bookmark__obj"):

                # img
                try:
                    img = element.find('div', class_="lazy-load-container js-lazy-loading")['data-src']
                except TypeError:
                    img = None

                # titles
                titles = []
                for i in element.find_all('span'):
                    if not i.has_attr('class'):
                        titles.append(i.text.strip())
                    else:
                        break

                # пошаговые рецепты, определенная кухня и название
                rec = re.compile(r'рецепты')
                kitchen = re.compile(r'кухня')
                deserts = re.compile(r'десерты')

                stepbystep = 0
                cuisine = None
                for title in titles:
                    if rec.search(title):
                        stepbystep = 1

                    if kitchen.search(title):
                        cuisine = title
                title = titles[-1]

                # number of portions
                try:
                    portions = element.find('span', class_='js-portions-count-print')\
                        .text.strip().split(' ')[0]
                except AttributeError:
                    portions = None

                # preparation time
                try:
                    time = element.find('span', class_='prep-time').text.strip()
                except AttributeError:
                    time = None

                # ingredients
                ingredients = []
                for i in element.find_all('p', class_='ingredients-list__content-item content-item js-cart-ingredients'):
                    ingredients.append(i['data-ingredient-object'])


                # link
                link = 'https://eda.ru'+element.find('a', class_='ingredients-list__link')['href']

                # favourited
                try:
                    favourite = element.find('div', class_='widget-list__totals')\
                        .find('span', class_='widget-list__favorite-count tooltip js-tooltip').text.strip()
                except Exception:
                    favourite = 0

                # likes
                try:
                    likes = element.find('div', class_='widget-list__totals')\
                        .find('span', class_='widget-list__like-count').text.strip()
                except Exception:
                    likes = 0

                # dislikes
                try:
                    dislikes = element.find('div', class_='widget-list__totals')\
                        .find('span', class_='widget-list__like-count widget-list__like-count_dislike').text.strip()
                except Exception:
                    dislikes = 0

                writer.writerow([img, title, link, stepbystep, cuisine,
                                portions, time, ingredients, favourite, likes, dislikes])
    return "finished"

In [6]:
getInfo_edaru('edaru_breakfast.csv', 'https://eda.ru/recepty/zavtraki', 1, 130)

'finished'